In [1]:
import torch
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

%load_ext autoreload
%autoreload 2

from deep_cnn_structures import ResNet

In [2]:
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, 
                                       transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True, 
                                          num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True,
                                      transform=transform)
testloader = DataLoader(testset, batch_size=16, shuffle=False,
                                         num_workers=2)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
net = ResNet()
net.cuda()

ResNet(
  (conv1): Conv2d (3, 32, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
  (layer1): Bottleneck(
    (conv1): Conv2d (32, 16, kernel_size=(1, 1), stride=(1, 1))
    (conv2): Conv2d (16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv3): Conv2d (16, 32, kernel_size=(1, 1), stride=(1, 1))
    (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
    (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
    (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
  )
  (conv2): Conv2d (32, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (layer2): Bottleneck(
    (conv1): Conv2d (64, 32, kernel_size=(1, 1), stride=(1, 1))
    (conv2): Conv2d (32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv3): Conv2d (32, 64, kernel_size=(1, 1), stride=(1, 1))
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0

In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters())

In [9]:
for epoch in range(15):
    running_loss = 0.
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
        optimizer.zero_grad()
        
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss = loss.data[0]
        
        if (i+1) % 1000 == 0:
            print('[%d, %5d] loss: %.3f' % (epoch+1, i+1, running_loss/2000))
            running_loss = 0.

print('Finished Training')

[1,  1000] loss: 0.000
[1,  2000] loss: 0.000
[1,  3000] loss: 0.000
[2,  1000] loss: 0.000
[2,  2000] loss: 0.000
[2,  3000] loss: 0.000
[3,  1000] loss: 0.000
[3,  2000] loss: 0.000
[3,  3000] loss: 0.000
[4,  1000] loss: 0.000
[4,  2000] loss: 0.000
[4,  3000] loss: 0.000
[5,  1000] loss: 0.000
[5,  2000] loss: 0.000
[5,  3000] loss: 0.000
[6,  1000] loss: 0.000
[6,  2000] loss: 0.000
[6,  3000] loss: 0.000
[7,  1000] loss: 0.000
[7,  2000] loss: 0.000
[7,  3000] loss: 0.000
[8,  1000] loss: 0.000
[8,  2000] loss: 0.000
[8,  3000] loss: 0.000
[9,  1000] loss: 0.000
[9,  2000] loss: 0.000
[9,  3000] loss: 0.000
[10,  1000] loss: 0.000
[10,  2000] loss: 0.000
[10,  3000] loss: 0.000
[11,  1000] loss: 0.000
[11,  2000] loss: 0.000
[11,  3000] loss: 0.000
[12,  1000] loss: 0.000
[12,  2000] loss: 0.000
[12,  3000] loss: 0.000
[13,  1000] loss: 0.000
[13,  2000] loss: 0.000
[13,  3000] loss: 0.000
[14,  1000] loss: 0.000
[14,  2000] loss: 0.000
[14,  3000] loss: 0.000
[15,  1000] loss: 0

In [12]:
correct = 0
total = 0
for data in testloader:
    images, labels = data
    outputs = net(Variable(images.cuda()))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Accuracy of the network on 10000 test images: %d %%' % (100 * correct / total))

Accuracy of the network on 10000 test images: 74 %


In [13]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

for data in testloader:
    images, labels = data
    outputs = net(Variable(images.cuda()))
    _, predicted = torch.max(outputs.data, 1)
    c = (predicted == labels.cuda())
    for i in range(16):
        label = labels[i]
        class_correct[label] += c[i]
        class_total[label] += 1
        
for i in range(10):
    print('Accuracy of %5s : %2d %%' % (classes[i], 100*class_correct[i]/class_total[i]))

Accuracy of plane : 74 %
Accuracy of   car : 88 %
Accuracy of  bird : 62 %
Accuracy of   cat : 61 %
Accuracy of  deer : 71 %
Accuracy of   dog : 65 %
Accuracy of  frog : 72 %
Accuracy of horse : 79 %
Accuracy of  ship : 81 %
Accuracy of truck : 83 %
